# Project - Segmenting and Clustering Neighborhoods in Toronto, Canada #
## Solving the problem 1 ##
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.  
To create the above dataframe:  
+ The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
+ Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
+ More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11**  in the above table.
+ If a cell has a borough but a **Not assigned**  neighborhood, then the neighborhood will be the same as the borough.
+ Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
+ In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.  
**Note.** I use pandas, the BeautifulSoup package, as a convenient way for me to convert the data in the table on the Wikipedia page to the aforementioned pandas data frame.

In [1]:
import pandas as pd
# !pip install requests
import requests
# !pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

In [2]:
#Scrape the List of postal codes of Canada
page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(page_url).text

In [3]:
soup = bs(source, 'html.parser')

In [4]:
#Get the first table in the html
data = soup.findAll('table')

In [5]:
def cleaner(seq, value):    
    temp = []                      
    for number in seq:
        if number not in value:
            temp.append(number)
    return temp

In [6]:
# create three lists to store the data into the table
list_postal_code = []
list_borough = []
list_neighborhood = []
# loading data into created lists
data = soup.find('table').find_all('tr')
for row in data:
    cells = row.find_all('td')
    if(len(cells) > 0):
            for cell in cells:
                lst = cleaner(cell.get_text(separator="<br />").strip().split("<br />"),
                              ['','\n','(',')',' / ',', '])
                txt = ''
                for n, value in enumerate(lst):
                    if n == 0 : list_postal_code.append(value)
                    elif n == 1 :list_borough.append(value)
                    else:
                        if len(txt) != 0 : txt += ', ' + value
                        else: txt = value
                if len(txt) == 0 : txt = 'Not assigned' 
                else: txt = txt.replace('(','').replace(')','').replace(' / ',', ').replace('  ',' ').replace(' ,',',').replace(',,','')
                list_neighborhood.append(txt)      

In [8]:
# create a new DataFrame from the three lists
df = pd.DataFrame({"PostalCode": list_postal_code,
                      "Borough": list_borough,
                 "Neighborhood": list_neighborhood})

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
df = df[df["Borough"] != "Not assigned"]

In [11]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [12]:
df_group = df.groupby("PostalCode")["Neighborhood"].apply(lambda x: "%s" % ', '.join(x))
df_group = df_group.reset_index(drop = False)
df_group.rename(columns = {"Neighborhood":"Neighborhood_joined"},inplace=True)

In [13]:
df_merge = pd.merge(df, df_group, on="PostalCode")
df_merge.drop(["Neighborhood"],axis=1,inplace=True)
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [14]:
df_merge.head()

,PostalCode,Borough,Neighborhood_joined
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [15]:
df_merge.drop_duplicates(inplace=True)

In [16]:
df_merge.head()

,PostalCode,Borough,Neighborhood_joined
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [17]:
df_merge.shape

(103, 3)

## **Solving the Problem 2** ##
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.   
Use the Geocoder package or the csv file to create the dataframe with the fields "PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"

In [18]:
def geocode(postal_code):
    x_y_coords = None
    while (x_y_coords is None):
        g = geocoders.google("{}, Toronto, Ontario".format(postal_code))
        x_y_coords = g.latlng
    latitude = x_y_coords[0]
    longitude = x_y_coords[1]
    return latitude, longitude

In [19]:
geo_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

In [20]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
geo_df.rename(columns = {"Postal Code":"PostalCode"},inplace = True)
geo_merged = pd.merge(geo_df, df_merge, on = "PostalCode")

In [22]:
geo_merged.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
geo_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [23]:
geo_data = geo_merged[["PostalCode","Borough","Neighborhood","Latitude","Longitude"]]

In [25]:
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


##### **Thank you !**
This notebook was created by *Dmitry Rabotko.*  
This notebook is part 2 of a course on **Coursera** called _Applied Data Science Capstone_ week 3. 